In [1]:
import fastf1
import pandas as pd

In [2]:
# Get historical data. Doesn't include 2025.
year = 2022
meetings_df = pd.DataFrame()
while year<2025:
    meeting = fastf1.get_event_schedule(year)
    meeting = meeting.loc[:,["RoundNumber","Country","Location","EventFormat"]]
    meeting['Year'] = year
    meetings_df = pd.concat([meetings_df,meeting],axis=0)
    year += 1



req         WARNING 	DEFAULT CACHE ENABLED! (814.73 MB) C:\Users\rohan\AppData\Local\Temp\fastf1


In [ ]:
for index,row in meetings_df.iterrows():
    round =


In [224]:
meetings_df

,RoundNumber,Country,Location,EventFormat,Year
0,0,Bahrain,Sakhir,testing,2022
1,1,Bahrain,Sakhir,conventional,2022
2,2,Saudi Arabia,Jeddah,conventional,2022
3,3,Australia,Melbourne,conventional,2022
4,4,Italy,Imola,sprint,2022
...,...,...,...,...,...
20,20,Mexico,Mexico City,conventional,2024
21,21,Brazil,São Paulo,sprint_qualifying,2024
22,22,United States,Las Vegas,conventional,2024
23,23,Qatar,Lusail,sprint_qualifying,2024


In [165]:
#meetings_df.loc[meetings_df["Year"]==2024,:]

In [130]:
event_sprint_2022 = fastf1.get_event(2022,4)
event_sprint_2022

RoundNumber                                                          4
Country                                                          Italy
Location                                                         Imola
OfficialEventName    FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...
EventDate                                          2022-04-24 00:00:00
EventName                                    Emilia Romagna Grand Prix
EventFormat                                                     sprint
Session1                                                    Practice 1
Session1Date                                 2022-04-22 13:30:00+02:00
Session1DateUtc                                    2022-04-22 11:30:00
Session2                                                    Qualifying
Session2Date                                 2022-04-22 17:00:00+02:00
Session2DateUtc                                    2022-04-22 15:00:00
Session3                                                    Practice 2
Sessio

In [131]:
event_sprint_2023 = fastf1.get_event(2023,4)
event_sprint_2023

RoundNumber                                             4
Country                                        Azerbaijan
Location                                             Baku
OfficialEventName    FORMULA 1 AZERBAIJAN GRAND PRIX 2023
EventDate                             2023-04-30 00:00:00
EventName                           Azerbaijan Grand Prix
EventFormat                               sprint_shootout
Session1                                       Practice 1
Session1Date                    2023-04-28 13:30:00+04:00
Session1DateUtc                       2023-04-28 09:30:00
Session2                                       Qualifying
Session2Date                    2023-04-28 17:00:00+04:00
Session2DateUtc                       2023-04-28 13:00:00
Session3                                  Sprint Shootout
Session3Date                    2023-04-29 12:30:00+04:00
Session3DateUtc                       2023-04-29 08:30:00
Session4                                           Sprint
Session4Date  

In [5]:
event_sprint_2024 = fastf1.get_event(2024,6)
event_sprint_2024

RoundNumber                                                   6
Country                                           United States
Location                                                  Miami
OfficialEventName    FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024
EventDate                                   2024-05-05 00:00:00
EventName                                      Miami Grand Prix
EventFormat                                   sprint_qualifying
Session1                                             Practice 1
Session1Date                          2024-05-03 12:30:00-04:00
Session1DateUtc                             2024-05-03 16:30:00
Session2                                      Sprint Qualifying
Session2Date                          2024-05-03 16:30:00-04:00
Session2DateUtc                             2024-05-03 20:30:00
Session3                                                 Sprint
Session3Date                          2024-05-04 12:00:00-04:00
Session3DateUtc                         

In [202]:
#how to get session from event 
qual = fastf1.get_session(2024,6,'Q')
qual.load()

race = fastf1.get_session(2024,6,'R')
race.load()

sprint = fastf1.get_session(2024,6,'S')
sprint.load()

core           INFO 	Loading data for Miami Grand Prix - Qualifying [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	D

In [209]:
#For EventFormat == sprint_shootout or EventFormat == sprint_qualifying

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

#For Race 
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"

In [175]:
# For EventFormat == sprint

## note its "S" for sprint

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"


In [ ]:
#For EventFormat == conventional

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

merged_qual_race_df["Sprint_Qual_Position"] = None
merged_qual_race_df["Sprint_Race_Position"] = None
merged_qual_race_df["Event_Type"] = "Race"

In [3]:
def get_event_info(year,round_num,event_format):
    
    qual = fastf1.get_session(year,round_num,'Q')
    qual.load()
    race = fastf1.get_session(year,round_num,'R')
    race.load()
    sprint = fastf1.get_session(year,round_num,'S')
    sprint.load()

    # For Qualifying 
    # data from session.results
    qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
    qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

    # For Race
    # data from session.results
    race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

    # data from session.laps
    race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
    final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
        'Sector1Time': 'mean',
        'Sector2Time': 'mean',
        'Sector3Time': 'mean',
        'SpeedST': 'mean',
        'Stint': 'max'
    }).reset_index()

    # outer join dfs --> complete data for a single race session 
    merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

    # Merge qual and race data 
    merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
    merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

    if event_format == "conventional":
        merged_qual_race_df["Sprint_Qual_Position"] = None
        merged_qual_race_df["Sprint_Race_Position"] = None
        merged_qual_race_df["Event_Type"] = "Race"

    elif event_format == "sprint":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    elif event_format == "sprint_shootout" or event_format == "sprint_qualifying":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    else:
        raise Exception(f"Unknown event format: {event_format}")
    
    return merged_qual_race_df

In [6]:
df = get_event_info(2024,6,event_sprint_2024["EventFormat"])

core           INFO 	Loading data for Miami Grand Prix - Qualifying [v3.5.3]


req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '4', '81', '63', '44', '27', '22', '18', '10', '31', '23', '14', '77', '2', '3', '20', '24']
core           INFO 	Loading data for Miami Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req   

In [7]:
df

,DriverNumber,Qual_Position,BroadcastName,TeamName,Race_Position,Sector1Time,Sector2Time,Sector3Time,SpeedST,Stint,Sprint_Qual_Position,Sprint_Race_Position,Event_Type
0,1,1.0,M VERSTAPPEN,Red Bull Racing,2.0,32.817607,36.208860,26.713333,303.035088,2.0,1.0,1.0,Sprint
1,10,12.0,P GASLY,Alpine,12.0,32.898643,36.625035,26.747912,316.333333,2.0,16.0,9.0,Sprint
2,11,4.0,S PEREZ,Red Bull Racing,4.0,33.070143,36.100632,26.638298,307.543860,3.0,3.0,3.0,Sprint
3,14,15.0,F ALONSO,Aston Martin,9.0,32.820071,36.642316,26.677316,307.052632,2.0,8.0,17.0,Sprint
4,16,2.0,C LECLERC,Ferrari,3.0,32.854107,36.211579,26.695088,308.192982,2.0,2.0,2.0,Sprint
5,18,11.0,L STROLL,Aston Martin,17.0,33.141232,36.573035,26.581158,312.122807,3.0,7.0,19.0,Sprint
6,2,17.0,L SARGEANT,Williams,20.0,32.971731,35.826444,26.601889,317.555556,2.0,18.0,10.0,Sprint
7,20,19.0,K MAGNUSSEN,Haas F1 Team,19.0,33.642286,36.072158,26.726526,316.140351,4.0,14.0,18.0,Sprint
8,22,10.0,Y TSUNODA,RB,7.0,32.928982,36.370719,26.671351,309.035088,2.0,15.0,8.0,Sprint
9,23,14.0,A ALBON,Williams,18.0,33.407179,36.616123,26.804368,320.719298,3.0,20.0,13.0,Sprint
